 # Predicting Book Success

In [1]:

# Initial imports
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
#Import the dataset
data = Path('books_clean.csv')
df = pd.read_csv(data)

# We can see from the preview of the DataFrame that multiple variables (also called features), such as the isbn13,
#published_year, average_rating, num_pages, ratings_count, can be used to predict the outcome: whether a book will have a good 
#rating (1) or will not (0) based on the fact that an average rating below of 4.5 will not likely be successful

df.loc[df['Rating'] <= 4.2, 'Rating_Classification'] = 'Low_Rating' 
df.loc[df['Rating'] > 4.2, 'Rating_Classification'] = 'High_Rating'

df.reset_index(inplace=True, drop=True)
df.head()

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Rating_Classification
0,9780439358071,4.49,7.38,870,1996446,English,Big,Famous,Top_Category,Serie,High_Rating
1,9780316015844,3.59,2.10,498,4367341,English,Big,Other,Other,Serie,Low_Rating
2,9780345538376,4.59,21.15,1728,97731,English,Big,Famous,Top_Category,Serie,High_Rating
3,9780393978896,3.84,2.73,464,1041597,English,Big,Other,Top_Category,Other,Low_Rating
4,9780142437209,4.11,5.46,532,1328143,English,Big,Other,Top_Category,Other,Low_Rating


In [3]:
# We will check for the variables from all columns
df.dtypes

ISBN                       int64
Rating                   float64
Price                    float64
Pages                      int64
Rating Count               int64
Language_Dummy            object
Size_Dummy                object
Famous_Dummy              object
Categories_Dummy          object
Serie_Dummy               object
Rating_Classification     object
dtype: object

In [4]:
#Count the current NaN values from the dataframe
df.isnull().sum().sum()

0

In [5]:
#Drop all the NaN values
df.dropna(inplace=True)

In [6]:
#Recount all the NaN values to make sure they are dropped
df.isnull().sum().sum()

0

In [7]:
#Use method to convert String to int
def rating(x):
    if x == 'Low_Rating':
        return 0
    if x == 'High_Rating':
        return 1
    

In [8]:
#Apply the method to the rating_classification column
df['Rating_Classification'] = df['Rating_Classification'].apply(rating)

In [9]:
#Use method to convert String to int
def binary(x):
    if x == 'Other':
        return 0
    if x == 'English'or 'Big' or 'Famous' or 'Serie' or 'Top_Category':
        return 1

In [10]:
#Apply the method to the rating_classification column
df['Size_Dummy'] = df['Size_Dummy'].apply(binary)

In [11]:
df['Serie_Dummy'] = df['Serie_Dummy'].apply(binary)

In [12]:
df['Famous_Dummy'] = df['Famous_Dummy'].apply(binary)

In [13]:
df['Language_Dummy'] = df['Language_Dummy'].apply(binary)

In [14]:
df['Categories_Dummy'] = df['Categories_Dummy'].apply(binary)

In [15]:
#Create a new dataframe with the target variables
df.tail(10)

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Rating_Classification
1921,9780842360616,3.99,5.56,356,3967,1,1,1,1,1,0
1922,9780425206867,3.76,3.19,216,22369,1,1,1,1,1,0
1923,9780142302330,4.39,15.14,1008,504,1,1,1,1,0,1
1924,9780140437850,3.53,5.69,289,3471,1,1,0,1,0,0
1925,9780812975932,3.44,3.28,142,5028,1,1,0,1,0,0
1926,9780441731183,3.78,3.89,341,6092,1,1,1,1,1,0
1927,9780811213769,4.30,3.59,99,599,1,1,0,1,0,1
1928,9780553562606,4.08,3.12,518,5356,1,1,0,1,1,0
1929,9781400044870,4.02,3.59,320,4455,1,1,0,1,0,0
1930,9780553273861,3.64,3.52,291,580,1,1,1,1,0,0


##  Separate the Features (X) from the Target (y)

In [16]:
#The Outcome column is defined as y, or the target.
#X, or features, is created by dropping the Outcome column from the DataFrame.

y = df["Rating_Classification"]
X = df.drop(columns=["Rating_Classification","ISBN","Rating","Rating Count"])


 ## Split our data into training and testing

In [17]:

# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1448, 7)
(483, 7)
(1448,)
(483,)


In [18]:
# Creating StandardScaler instance (to scalate data)
scaler = StandardScaler()

In [19]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [20]:
# Scaling and trnsforming data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Fitting the Random-forest

In [21]:
# Create a random forest classifier. The n_estimators will allow us to set the number of trees that will be created by the algorithm
# The best practice is to use between 64 and 128 random forests
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [22]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making Predictions Using the Tree Model

In [23]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Model evaluation

In [24]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df



,Predicted 0,Predicted 1
Actual 0,361,23
Actual 1,81,18


In [25]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [26]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,361,23
Actual 1,81,18


Accuracy Score : 0.7846790890269151
Classification Report
              precision    recall  f1-score   support

           0       0.82      0.94      0.87       384
           1       0.44      0.18      0.26        99

    accuracy                           0.78       483
   macro avg       0.63      0.56      0.57       483
weighted avg       0.74      0.78      0.75       483



In [27]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.4555133 , 0.45921355, 0.00140104, 0.01836751, 0.01995219,
       0.02248519, 0.02306722])

In [28]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.4592135519182356, 'Pages'),
 (0.45551329705725374, 'Price'),
 (0.02306722445989212, 'Serie_Dummy'),
 (0.022485193005481453, 'Categories_Dummy'),
 (0.019952189641980558, 'Famous_Dummy'),
 (0.018367508006977833, 'Size_Dummy'),
 (0.0014010359101787133, 'Language_Dummy')]